In [1]:
!pip install datasets

In [2]:
# T5를 이용한 뉴스 요약 모델 학습 및 평가
import numpy as np # 과학 기술 계산을 위한 라이브러리인 NumPy를 불러옵니다.
from datasets import load_dataset # Hugging Face `datasets` 라이브러리에서 데이터셋을 불러오는 함수를 불러옵니다.

# argilla/news-summary 데이터셋을 불러와 'test' 분할을 사용합니다.
news = load_dataset("argilla/news-summary", split="test")

# 데이터셋을 pandas DataFrame으로 변환하고 5000개의 샘플을 랜덤하게 추출합니다.
df = news.to_pandas().sample(5000, random_state=42)[["text", "prediction"]]

# 'summarize: ' 접두사를 'text' 열에 추가하여 모델이 요약 작업임을 알 수 있게 합니다.
df["text"] = "summarize: " + df["text"]

# 'prediction' 열의 값을 리스트에서 첫 번째 텍스트 요소로 추출합니다.
df["prediction"] = df["prediction"].map(lambda x: x[0]["text"])

# 전체 데이터셋을 60% (학습), 20% (검증), 20% (테스트)로 분할합니다.
train, valid, test = np.split(
    df.sample(frac=1, random_state=42), [int(0.6 * len(df)), int(0.8 * len(df))])

# 분할된 데이터의 첫 번째 샘플과 데이터셋 크기를 출력합니다.
print(f"Source News : {train.text.iloc[0][:200]}")
print(f"Summarization : {train.prediction.iloc[0][:50]}")
print(f"Training Data Size : {len(train)}")
print(f"Validation Data Size : {len(valid)}")
print(f"Testing Data Size : {len(test)}")


Source News : summarize: DANANG, Vietnam (Reuters) - Russian President Vladimir Putin said on Saturday he had a normal dialogue with U.S. leader Donald Trump at a summit in Vietnam, and described Trump as civil, we
Summarization : Putin says had useful interaction with Trump at Vi
Training Data Size : 3000
Validation Data Size : 1000
Testing Data Size : 1000


/opt/conda/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [3]:
!pip install sentencepiece

In [5]:
# T5 모델 학습을 위한 데이터셋과 데이터로더를 준비합니다.
import torch # 딥러닝 프레임워크인 PyTorch를 불러옵니다.
from transformers import T5Tokenizer # T5 모델의 토크나이저를 불러옵니다.
from torch.utils.data import TensorDataset, DataLoader # PyTorch의 데이터셋과 데이터로더 클래스를 불러옵니다.
from torch.utils.data import RandomSampler, SequentialSampler # 데이터 샘플러를 불러옵니다.
from torch.nn.utils.rnn import pad_sequence # 시퀀스 패딩 유틸리티를 불러옵니다.

# DataFrame 데이터를 T5 모델이 처리할 수 있는 형식의 데이터셋으로 변환하는 함수를 정의합니다.
def make_dataset(data, tokenizer, device):
    # 입력 텍스트를 토큰화하고 패딩, 자르기를 수행합니다.
    source = tokenizer(
        text=data.text.tolist(),
        padding="max_length", # 최대 길이로 패딩합니다.
        max_length=128, # 최대 시퀀스 길이를 128로 설정합니다.
        #pad_to_max_length=True, # 최대 길이로 패딩을 강제합니다.
        truncation=True, # 최대 길이를 초과하는 텍스트를 자릅니다.
        return_tensors="pt" # PyTorch 텐서 형식으로 반환합니다.
    )

    # 목표(요약) 텍스트를 토큰화하고 패딩, 자르기를 수행합니다.
    target = tokenizer(
        text=data.prediction.tolist(),
        padding="max_length", 
        max_length=128,
        #pad_to_max_length=True,
        truncation=True,
        return_tensors="pt"
    )
    
    # 생성된 텐서들을 지정된 디바이스(CPU/GPU)로 옮깁니다.
    source_ids = source["input_ids"].squeeze().to(device)
    source_mask = source["attention_mask"].squeeze().to(device)
    target_ids = target["input_ids"].squeeze().to(device)
    target_mask = target["attention_mask"].squeeze().to(device)
    # 텐서들로 구성된 TensorDataset 객체를 반환합니다.
    return TensorDataset(source_ids, source_mask, target_ids, target_mask)

# 데이터셋과 샘플러를 이용하여 데이터로더를 생성하는 함수를 정의합니다.
def get_datalodader(dataset, sampler, batch_size):
    # 주어진 샘플러를 사용하여 데이터를 샘플링합니다.
    data_sampler = sampler(dataset)
    # 데이터셋과 샘플러, 배치 크기를 사용하여 데이터로더를 생성합니다.
    dataloader = DataLoader(dataset, sampler=data_sampler, batch_size=batch_size)
    return dataloader

# 학습에 사용될 하이퍼파라미터 및 설정을 정의합니다.
epochs = 5
batch_size = 8
device = "cuda" if torch.cuda.is_available() else "cpu" # GPU 사용 가능 여부에 따라 디바이스를 설정합니다.

# 't5-small' 사전 학습 모델의 토크나이저를 불러옵니다.
tokenizer = T5Tokenizer.from_pretrained(
    pretrained_model_name_or_path="t5-small")

# 학습, 검증, 테스트 데이터셋을 생성합니다.
train_dataset = make_dataset(train, tokenizer, device)
# 학습 데이터로더를 생성합니다. (랜덤 샘플링)
train_dataloader = get_datalodader(train_dataset, RandomSampler, batch_size)
valid_dataset = make_dataset(valid, tokenizer, device)
# 검증 데이터로더를 생성합니다. (순차적 샘플링)
valid_dataloader = get_datalodader(valid_dataset, SequentialSampler, batch_size)
test_dataset = make_dataset(test, tokenizer, device)
# 테스트 데이터로더를 생성합니다. (순차적 샘플링)
test_dataloader = get_datalodader(test_dataset, SequentialSampler, batch_size)

# 학습 데이터로더에서 첫 번째 배치를 출력하여 데이터 형식을 확인합니다.
print(next(iter(train_dataloader)))
# 특정 토큰 ID를 실제 토큰으로 변환하여 확인합니다.
print(tokenizer.convert_ids_to_tokens(21603))
print(tokenizer.convert_ids_to_tokens(10))

[tensor([[21603,    10,    41,  ...,     0,     0,     0],
        [21603,    10,   549,  ...,  4828,  8286,     1],
        [21603,    10,   549,  ...,    12,    80,     1],
        ...,
        [21603,    10,   445,  ...,     0,     0,     0],
        [21603,    10,  6554,  ...,   471,     6,     1],
        [21603,    10,    41,  ...,  2876,  2721,     1]], device='cuda:0'), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'), tensor([[    3, 15038,   667,  ...,     0,     0,     0],
        [22439, 22121,     7,  ...,     0,     0,     0],
        [27463, 22121,     7,  ...,     0,     0,     0],
        ...,
        [12605,    29,  2095,  ...,     0,     0,     0],
        [  412,     5,   134,  ...,     0,     0,     0],
        [11543,  2689,    10,  ...,     0,     0,     0]], device='cuda:0'), ten

In [6]:
# T5 모델과 옵티마이저를 설정합니다.
from torch import optim # PyTorch의 최적화 함수를 불러옵니다.
from transformers import T5ForConditionalGeneration # T5 모델 클래스를 불러옵니다.

# 't5-small' 사전 학습 모델을 불러와 지정된 디바이스로 옮깁니다.
model = T5ForConditionalGeneration.from_pretrained(
    pretrained_model_name_or_path="t5-small",).to(device)
# AdamW 옵티마이저를 사용하여 모델의 파라미터를 최적화하도록 설정합니다.
optimizer = optim.AdamW(model.parameters(), lr=1e-5, eps=1e-8)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
# 모델 학습 및 평가에 필요한 함수들을 정의합니다.
import numpy as np # NumPy를 다시 불러옵니다.
from torch import nn # PyTorch의 신경망 모듈을 불러옵니다.
from tqdm.auto import tqdm # tqdm 라이브러리를 불러와 진행 상태 바를 표시합니다.

# 예측 결과와 레이블을 비교하여 정확도를 계산하는 함수입니다.
def calc_accuracy(preds, labels):
    # 예측 텐서를 평탄화하고 가장 높은 값의 인덱스를 취합니다.
    pred_flat = np.argmax(preds, axis=1).flatten()
    # 레이블 텐서를 평탄화합니다.
    labels_flat = labels.flatten()
    # 예측과 레이블이 일치하는 비율을 계산하여 반환합니다.
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 모델 학습을 위한 함수입니다.
def train(model, optimizer, dataloader):
    model.train() # 모델을 학습 모드로 설정합니다.
    train_loss = 0.0

    # 데이터로더로부터 배치를 반복하여 가져옵니다.
    for source_ids, source_mask, target_ids, target_mask in tqdm(dataloader, desc="Training"):
        # 디코더 입력은 목표 텐서의 첫 번째 토큰을 제외한 부분입니다.
        decoder_input_ids = target_ids[:, :-1].contiguous()
        # 레이블은 목표 텐서의 첫 번째 토큰을 제외한 부분입니다.
        labels = target_ids[:, 1:].clone().detach()
        # 패딩 토큰 ID를 -100으로 변경하여 손실 계산에서 제외합니다.
        labels[target_ids[:, 1:] == tokenizer.pad_token_id] = -100

        # 모델에 입력을 전달하고 출력을 계산합니다.
        outputs = model(
            input_ids=source_ids,
            attention_mask=source_mask,
            decoder_input_ids=decoder_input_ids,
            labels=labels,
        )

        loss = outputs.loss # 손실 값을 가져옵니다.
        train_loss += loss.item() # 손실을 누적합니다.

        optimizer.zero_grad() # 옵티마이저의 기울기를 초기화합니다.
        loss.backward() # 손실에 대한 역전파를 수행합니다.
        optimizer.step() # 옵티마이저를 업데이트하여 모델의 가중치를 수정합니다.

    train_loss = train_loss / len(dataloader) # 평균 학습 손실을 계산합니다.
    return train_loss

# 모델 검증을 위한 함수입니다.
def evaluation(model, dataloader):
    with torch.no_grad(): # 기울기 계산을 비활성화하여 메모리를 절약하고 속도를 높입니다.
        model.eval() # 모델을 평가 모드로 설정합니다.
        val_loss = 0.0

        # 데이터로더로부터 배치를 반복하여 가져옵니다.
        for source_ids, source_mask, target_ids, target_mask in tqdm(dataloader, desc="Validation"):
            # 디코더 입력과 레이블을 학습 시와 동일하게 설정합니다.
            decoder_input_ids = target_ids[:, :-1].contiguous()
            labels = target_ids[:, 1:].clone().detach()
            labels[target_ids[:, 1:] == tokenizer.pad_token_id] = -100

            # 모델에 입력을 전달하고 출력을 계산합니다.
            outputs = model(
                input_ids=source_ids,
                attention_mask=source_mask,
                decoder_input_ids=decoder_input_ids,
                labels=labels,
            )

            loss = outputs.loss
            val_loss += loss.item()

    val_loss = val_loss / len(dataloader) # 평균 검증 손실을 계산합니다.
    return val_loss

# 모델 학습 및 검증 루프입니다.
best_loss = 10000 # 가장 낮은 검증 손실을 저장하기 위한 변수를 초기화합니다.
for epoch in range(epochs):
    # 학습 함수를 호출하여 한 에포크 동안 학습을 수행합니다.
    train_loss = train(model, optimizer, train_dataloader)
    # 검증 함수를 호출하여 한 에포크 동안 검증을 수행합니다.
    val_loss = evaluation(model, valid_dataloader)
    # 현재 에포크의 학습 및 검증 손실을 출력합니다.
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f} Val Loss: {val_loss:.4f}")

    # 현재 검증 손실이 이전까지의 최소 손실보다 작으면 모델을 저장합니다.
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), "../models/T5ForConditionalGeneration.pt")
        print("Saved the model weights")

Training:   0%|          | 0/375 [00:00<?, ?it/s]

Validation:   0%|          | 0/125 [00:00<?, ?it/s]

Epoch 1: Train Loss: 4.3381 Val Loss: 3.3380
Saved the model weights


Training:   0%|          | 0/375 [00:00<?, ?it/s]

Validation:   0%|          | 0/125 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/125 [00:00<?, ?it/s]

Epoch 4: Train Loss: 2.9997 Val Loss: 2.6775
Saved the model weights


Training:   0%|          | 0/375 [00:00<?, ?it/s]

Validation:   0%|          | 0/125 [00:00<?, ?it/s]

Epoch 5: Train Loss: 2.9026 Val Loss: 2.6135
Saved the model weights


In [8]:
# 테스트 데이터에 대한 요약 생성을 시작합니다.
model.eval() # 모델을 평가 모드로 설정합니다.
with torch.no_grad(): # 기울기 계산을 비활성화합니다.
    # 테스트 데이터로더에서 배치를 가져옵니다.
    for source_ids, source_mask, target_ids, target_mask in test_dataloader:
        # 모델의 `generate` 함수를 사용하여 요약 텍스트를 생성합니다.
        generated_ids = model.generate(
            input_ids=source_ids,
            attention_mask=source_mask,
            max_length=128, # 생성될 시퀀스의 최대 길이를 설정합니다.
            num_beams=3, # 빔 서치(beam search)의 크기를 3으로 설정합니다.
            repetition_penalty=2.5, # 반복 패널티를 설정합니다.
            length_penalty=1.0, # 길이 패널티를 설정합니다.
            early_stopping=True, # 조기 종료를 활성화합니다.
        )

        # 생성된 요약과 실제 요약을 비교하고 출력합니다.
        for generated, target in zip(generated_ids, target_ids):
            # 생성된 토큰 ID를 텍스트로 디코딩합니다.
            pred = tokenizer.decode(
                generated, skip_special_tokens=True, clean_up_tokenization_spaces=True
            )
            # 실제 토큰 ID를 텍스트로 디코딩합니다.
            actual = tokenizer.decode(
                target, skip_special_tokens=True, clean_up_tokenization_spaces=True
            )
            print("Generated Headline Text:", pred) 
            print("Actual Headline Text   :", actual) 
        break # 첫 번째 배치만 처리하고 루프를 종료합니다.

Generated Headline Text: Clinton leads Trump by 4 percentage points in four-war race for Nov. 8 election
Actual Headline Text   : Clinton leads Trump by 4 points in Washington Post: ABC News poll
Generated Headline Text: U.S. senators sharpen potential line of attack against Gorsuch's nomination to Supreme Court
Actual Headline Text   : Democrats question independence of Trump Supreme Court nominee
Generated Headline Text: U.S. warns Saudi Arabia over humanitarian situation in Yemen could constrain U.S. aid, a U.S. official says
Actual Headline Text   : In push for Yemen aid, U.S. warned Saudis of threats in Congress
Generated Headline Text: Romanian anti-corruption prosecutors open investigation into Liviu Dragnea on suspicion of forming criminal group to siphon off cash from state projects
Actual Headline Text   : Romanian ruling party leader investigated over 'criminal group'
Generated Headline Text: environmental activist endorsed Hillary Clinton for U.S. president
Actual Headline 